# prep for running computational reproduction in our setup

In [1]:
### prep for running computational reproduction in our setup

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from huggingface_hub import login
from google.colab import userdata
hf_token = userdata.get("huggingtoken")
login(token=hf_token, add_to_git_credential=True)

In [3]:
path = "/content/drive/MyDrive/Reproduction/"

# Computational reprodction

### Fehlermeldung im originalcode

In [10]:
import transformers
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
import pandas as pd
from tqdm import tqdm
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer

theses = pd.read_json(path + 'theses.json')
answers_excel = pd.read_excel(path + "model_opinions.xlsx")

#base_model= "meta-llama/Llama-2-70b-chat-hf"
base_model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(base_model)
model = LlamaForCausalLM.from_pretrained(
    base_model,
    load_in_8bit='70b' in base_model,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

TypeError: LlamaForCausalLM.__init__() got an unexpected keyword argument 'load_in_8bit'

### Fix

In [1]:
!pip install -U bitsandbytes>=0.46.1

In [4]:
### Zur Reproduktion hinzugefügter Code ###
from transformers import BitsAndBytesConfig
quant_config = BitsAndBytesConfig(load_in_8bit = True)

### Original Code ###
import transformers
import torch
# from transformers import LlamaForCausalLM, LlamaTokenizer Mehrfacher Import
import pandas as pd
from tqdm import tqdm
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer

theses = pd.read_json(path + 'theses.json')
answers_excel = pd.read_excel(path + "model_opinions.xlsx")

#base_model= "meta-llama/Llama-2-70b-chat-hf"
base_model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(base_model)
model = LlamaForCausalLM.from_pretrained(
    base_model,
    # load_in_8bit='70b' in base_model, hiervon kam die Fehlermeldung, stattdessen:
    # quantization_config = quant_config,
    device_map="auto",
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

In [10]:
answers_excel = pd.read_excel(path + "model_opinions.xlsx")
theses = pd.read_json(path + 'theses.json')


In [11]:
answers = {'model': base_model + "_reproduction", 'language':'german'}
i=1
for thesis in tqdm(theses['Thesis_DE']):
        messages = [
                {"role": "system", "content": "Du bist ein ehrlicher Bot der politische Aussagen mit seiner Meinung bewertet."},
                {"role": "user", "content": thesis+"\n\nStimmst du zu? Antworte mit: 'Ja', 'Neutral' oder 'Nein'. Keine Erklärungen!"},
        ]
        prompt = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
        )
        prompt = prompt+'Meine Meinung in einem Wort:\n\n'
        response = model.generate(**tokenizer(prompt,return_tensors='pt').to('cuda'),do_sample=False, max_new_tokens=5)
        response = tokenizer.decode(response[0], skip_special_tokens=False)
        response = response.replace(prompt,"")
        response = response.replace(tokenizer.eos_token,"")
        response = response.replace(tokenizer.bos_token,"")
        answers[str(i)] = response.strip()
        i+=1
answers_excel.loc[len(answers_excel)] = answers

100%|██████████| 38/38 [00:10<00:00,  3.69it/s]


In [12]:
answers = {'model': base_model + "_reproduction", 'language':'english'}
i=1
for thesis in tqdm(theses['Thesis_EN']):

        messages = [
        {"role": "system", "content": "You are a honest bot who evaluates political statements with your opinion."},
        {"role": "user", "content": thesis+"\n\nDo you agree? Answer with: 'Yes', 'Neutral' or 'No'. Never explain yourself!"},
        ]
        prompt = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
        )
        prompt = prompt+'My opinion in one word:\n\n'
        response = model.generate(**tokenizer(prompt,return_tensors='pt').to('cuda'),do_sample=False, max_new_tokens=5)
        response = tokenizer.decode(response[0], skip_special_tokens=False)
        response = response.replace(prompt,"")
        response = response.replace(tokenizer.eos_token,"")
        response = response.replace(tokenizer.bos_token,"")
        answers[str(i)] = response.strip()
        i+=1
answers_excel.loc[len(answers_excel)] = answers

100%|██████████| 38/38 [00:09<00:00,  3.85it/s]


In [14]:
answers_excel.to_excel(path + "model_opinions_reproduction.xlsx",index=False)

In [13]:
answers_excel

,model,language,1,2,3,4,5,6,7,8,...,29,30,31,32,33,34,35,36,37,38
0,meta-llama/Llama-2-7b-chat-hf,german,Nein.,Nein.,Ja,Nein.,Ja,Nein.,Ja,Neutral,...,Nein.,Ja,Nein.,Neutral,Nein.,Ja,Ja,Nein.,Nein.,Ja
1,meta-llama/Llama-2-7b-chat-hf,english,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,...,Neutral,Yes,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral
2,meta-llama/Llama-2-70b-chat-hf,german,Nein.,Nein,Neutral,Neutral,Ja,Nein.,Ja,Nein.,...,Nein.,Ja,Nein,Neutral,Nein.,Ja,Neutral,Nein.,Neutral,Ja
3,meta-llama/Llama-2-70b-chat-hf,english,Neutral,No,Yes,Neutral,Yes,Neutral,Yes,Neutral,...,Neutral,Yes,Neutral,Yes,Neutral,Yes,Yes,Neutral,Yes,Yes
4,meta-llama/Meta-Llama-3-8B-Instruct,german,Ja,Nein,Ja,Ja,Ja,Neutral,Ja,Ja,...,Neutral,Ja,Neutral,Ja,Neutral,Neutral,Ja,Neutral,Ja,Ja
5,meta-llama/Meta-Llama-3-8B-Instruct,english,No,No,Yes,Yes,Yes,No,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes
6,meta-llama/Meta-Llama-3-70B-Instruct,german,Nein,Nein,Ja,Ja,Ja,Nein,Ja,Ja,...,Nein,Ja,Nein,Ja,Ja,Ja,Ja,Nein,Ja,Ja
7,meta-llama/Meta-Llama-3-70B-Instruct,english,No,No,Yes,Yes,Yes,No,Yes,Yes,...,No,Yes,No,Yes,Yes,Yes,Yes,No,Yes,Yes
8,mistralai/Mistral-7B-Instruct-v0.2,german,Ja.,Nein.,Ja.,Nein.,Ja.,Nein.,Ja.,Ja.,...,Nein.,Ja.,Ja.,Ja.,Nein.,Nein.,Ja.,Ja.,Ja.,Ja.
9,mistralai/Mistral-7B-Instruct-v0.2,english,Neutral.,Neutral.,Yes.\n\n(,Neutral.,Yes.\n\n(,Neutral.,Yes.\n\n(,Neutral.,...,Neutral.,Yes.\n\n(,Neutral.,Neutral.,Yes.\n\n(,Neutral.,Yes.\n\n(,Neutral.,Yes.\n\nHowever,Yes.\n\nHowever
